# Final Project

## Setup configuration for Using Fabric Testbed via FABlib API

The following notebook can be used to configure your Jupyter environment.  The product of running this notebook are the following:

- `fabric_rc`:  File used to configure a FABlib application.  
- `fabric_bastion_key`: Fabric Bastion key pair. In order to minimize security incidents on FABRIC, access to VMs and other resources administered by users is controlled using a bastion host. You will need to set up an ssh keypair that will be used to jump through the bastion host to your VMs and other resources. This keypair is unique to you and is only used to set up ssh proxy connections through the bastion host to your FABRIC resources. More information about how to access your experiment through the bastion host can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
- `slice_key` and `slice_key.pub`: Sliver Key pair.
- `ssh_config`: File used to ssh from from a terminal to FABRIC VM by jumping through the FABRIC bastion host. 

## Set Project ID and generate the configuration

Edit the following cell by entering your Project ID for the FABRIC Project to use in your Jupyter container.  

- The Project ID can be from any of your projects. The ID can be found in the 'Basic Info' tab for each of the [projects](https://portal.fabric-testbed.net/experiments#projects) in the FABRIC portal.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Update this line to specify your project id
project_id = "0508639f-d515-44c8-b922-cf4cf000b6d9"

# Uncomment the line below if using 'FABRIC Tutorials' Project
#project_id="a7818636-1fa1-4e77-bb03-d171598b0862"

fablib = fablib_manager(project_id=project_id)

## Display the configuration

In [ ]:
fablib.show_config();

## Validate the configuration;
- Checks the validity of the bastion keys and regenerates them if they are expired
- Generates Sliver keys if they do not exist already

In [ ]:
fablib.verify_and_configure()

## Save the configuration for subsequent use

In [ ]:
fablib.save_config()

## Set up the Experiment
In this section you will use the Fablib manager to create a new slice that is composed of 2 nodes for the Ping Assignment.

## Reserve Resources
In the 'EDUKY' site, we will reserve a set of 2 nodes arranged in a line, with one node designated as a server and the other as a client. Each node will have the following specifications: 1 CPU core, 2GB of RAM, and 10GB of storage capacity. Each node will have 1 network card (NICs) to communicate with each other. All nodes will be preloaded with an 'Ubuntu' Linux OS. Upon submission, the slice will be named 'MyLayer2PingSlice'.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "MyLayer2PingSlice"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    #Create Slice
    slice = fablib.new_slice(slice_name)

    # Add node
    client = slice.add_node(name="client", site=site)
    client.set_capacities(cores=cores, ram=ram, disk=disk)
    client.set_image(image)
    iface1 = client.add_component(model='NIC_Basic', name="client-nic1").get_interfaces()[0]
    
    # Add node
    server = slice.add_node(name="server", site=site)
    server.set_capacities(cores=cores, ram=ram, disk=disk)
    server.set_image(image)
    iface2 = server.add_component(model='NIC_Basic', name="server-nic1").get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name="bridge1", interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit()
    
except Exception as e:
    print(f"Slice Failed: {e}")

## Set up the Experiment Network
The network configuration part of this slice will connect the nodes with specific IPs such that the nodes are able to communicate with each other. The slice will use the network name to find the corresponding devices (NIC cards) used in the slice creation stage so that we can provide the correct connection in the network, in this case, "bridge1".

This slice contains a single subnetwork in IPv4: 10.20.30.0/24 (between server and the client, "bridge1"). This cell uses the alternative "cidr" to set the subnet based on the specified number of bits. This slice will provide IP 10.20.30.40 to the client node and 10.20.30.41 to the server node following the guidelines set by the network subnet.

After providing the IPs to the corresponding devices, the cell will verify that all interfaces are up and execute "ip a" to output the connections that are active. Then, we will add the additional route from the client to the server in case the route is not added correctly.

In [ ]:
# Setup Network
try:
    client_name = "client"
    server_name = "server"
    network_service_name = "bridge1"

    client_ip = '10.20.30.40'
    server_ip = '10.20.30.41'
    
    client = slice.get_node(name=client_name)        
    iface1 = client.get_interface(network_name=network_service_name)  
    iface1.set_ip(ip=client_ip, cidr="24")
    
    iface1.get_device_name()
    
    client.execute("echo 100.20.30.40 client | sudo tee -a /etc/hosts")
    client.execute(f"sudo ip link set dev { iface1.get_device_name()} up")
    
    stdout, stderr = client.execute(f'ip addr show {iface1.get_os_interface()}')
    server = slice.get_node(name=server_name)        
    iface2 = server.get_interface(network_name=network_service_name)  
    iface2.set_ip(ip=server_ip, cidr="24")
    
    stdout, stderr = server.execute(f'ip addr show {iface2.get_os_interface()}')
    server.execute(f"sudo ip link set dev { iface2.get_device_name()} up")
 
    print("Printing Server's Network")
    out, err = server.execute("ip a ")
    
    print("Getting control interface ip")
    out, err = server.execute("ip a | grep inet | grep enp | awk '{print $2}'")
    ip = ((out.split('\n')[0]).split('/'))[0]
    print(f"Ip found: {ip}")
    
    print("Getting Client's interface name")
    print(f'Device Found: {iface1.get_device_name()}')
    
    # add route if needed
    client.execute(f"sudo ip route add {ip} dev {iface1.get_device_name()}")

except Exception as e:
    print(f"Error: {e}")

## Configure the software needed for the nodes in the experiment¶
In this section of the slice creation, we will add any additional software, tools or scripts that we need for our experiments. For this slice we don't need any special software nor additional configuration.

In [ ]:
# Install Software
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    print("No additional software Needed")
except Exception as e:
    print(f"Exception: {e}")

## Retrieve Slice

### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

try:
    slice_name = 'MyLayer2PingSlice'
    
    slice = fablib.get_slice(slice_name)
    slice.list_nodes()
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

### View Topology

#### Get the topology
To configure the Nodes we need to get the topology of the experiment by getting the slice component. In this tutorial we get the slice by the name given when the slice was created.
The folowing cell prints the topology.

In [ ]:
# Preview Slice node's attributes
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()                
except Exception as e:
    print(f"Fail: {e}")

## Cleanup Resources

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")